<a href="https://colab.research.google.com/github/SaguPandya96/Research-Project-Fall-2020/blob/master/Research_Project_ML_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Research Project**


**1. Import the libraries And Data**





In [2]:
!git clone https://github.com/facebookresearch/fastText.git

fatal: destination path 'fastText' already exists and is not an empty directory.


In [3]:
%cd fastText
!pip install .

/content/fastText
Processing /content/fastText
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3010555 sha256=0b32db36f3bfad996f34bf53637871d39b20e7034377078bad45ae260626f979
  Stored in directory: /tmp/pip-ephem-wheel-cache-auheclgr/wheels/a1/9f/52/696ce6c5c46325e840c76614ee5051458c0df10306987e7443
Successfully built fasttext
  Found existing installation: fasttext 0.9.2
    Uninstalling fasttext-0.9.2:
      Successfully uninstalled fasttext-0.9.2


In [5]:
import pandas as pd
import string

from textblob.classifiers import NaiveBayesClassifier as NBC
from textblob.classifiers import DecisionTreeClassifier
from sklearn.metrics import f1_score
import nltk
nltk.download('punkt')
nltk.download('stopwords')

import itertools
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.utils import shuffle
import numpy as np

'''
FastText requires specific training file format - see
https://github.com/facebookresearch/fastText for details.
'''
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import fasttext
from sklearn.utils import shuffle
from sklearn.metrics import f1_score


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**2. Import the Chromium Data**

In [7]:
#load data
DATA_PATH1 = '/content/drive/My Drive/Research Project/chromium.csv'
data1 = pd.read_csv(DATA_PATH1, sep='\t')
data_top1 = data1.head()
print("Chromium Data")
data_top1

Chromium Data


,title,description,type
0,Define a way to enforce performance constraint...,\nThere are cases where tests should fail if a...,Bug
1,Websites that don't work with Chrome Browser,\nChrome Version : 3.0.195.10\r\nURLs (i...,Compat
2,Redraw issue/truncation with confirmation item...,\nChromium 0.3.155.0 (Developer Build 3546)\r\...,Bug
3,Implement CanvasRenderingContext2D for access ...,\nApplications that do background image proces...,Feature
4,ath9k: panic in ath_tx_last_beacon + 0x29a,\nToT (3.0 kernel) panic'd on resume. Was ass...,Bug


**3. Import the Linux Bugs Data**

In [8]:
DATA_PATH2 = '/content/drive/My Drive/Research Project/linux_bugs_usage_ready.csv'
data2 = pd.read_csv(DATA_PATH2, sep='\t')
data_top2 = data2.head()
print("Linux Data")
data_top2

Linux Data


,title,message,importance,product,component
0,oops when using ide-cd with 2.5.45 and cdrecord,Please enter Exact Kernel version:2.5.45 Distr...,P2 normal,IO/Storage,IDE
1,NUMA-Q hangs during TSC initialization on boot.,Exact Kernel version: 2.5.46 Distribution: deb...,P2 normal,Platform Specific/Hardware,i386
2,Enabling shared pagetables causes KDE to wierd...,Exact Kernel version: 2.5.46-mm1 Distribution:...,P2 normal,Memory Management,Other
3,Dcache spirals out of control on 2.5.43-mm2,Exact Kernel version: 2.5.43-mm2 Distribution:...,P2 blocking,IO/Storage,Other
4,64GB highmem BUG(),Exact Kernel version: 2.5.40 Hardware Environm...,P2 normal,Memory Management,Other


**3. Merge title and message for chromium dataset**

In [9]:

def merge_title_and_message(data, message_col_name='message'):
  data['text'] = data['title'] + ' ' + data[message_col_name]
  data = data.drop(['title'], axis=1)
  data = data.drop([message_col_name], axis=1)
  return data
data1 = merge_title_and_message(data1, message_col_name='description')
data_top1 = data1.head()
print("Chromium Merge Title and Message Data")
data_top1

Chromium Merge Title and Message Data


,type,text
0,Bug,Define a way to enforce performance constraint...
1,Compat,Websites that don't work with Chrome Browser \...
2,Bug,Redraw issue/truncation with confirmation item...
3,Feature,Implement CanvasRenderingContext2D for access ...
4,Bug,ath9k: panic in ath_tx_last_beacon + 0x29a \nT...


**4. Merge title and message for Linux Bugs dataset**

In [10]:
data2 = merge_title_and_message(data2)
data_top2 = data2.head()
print("Linux Merge Title and Message Data")
data_top2

Linux Merge Title and Message Data


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops when using ide-cd with 2.5.45 and cdrecor...
1,P2 normal,Platform Specific/Hardware,i386,NUMA-Q hangs during TSC initialization on boot...
2,P2 normal,Memory Management,Other,Enabling shared pagetables causes KDE to wierd...
3,P2 blocking,IO/Storage,Other,Dcache spirals out of control on 2.5.43-mm2 Ex...
4,P2 normal,Memory Management,Other,64GB highmem BUG() Exact Kernel version: 2.5.4...


**5. Remove the Strip Punctuation For Both the Data**

In [11]:
def strip_punctuations(data, column_name='text'):
  '''
  Strips punctuations from the end of each token.
  This uses suggestion from https://stackoverflow.com/questions/34293875/how-to-remove-punctuation-marks-from-a-string-in-python-3-x-using-translate
  to accomplish this really fast.
  '''
  translator = str.maketrans('', '', string.punctuation)
  data['text'] = data['text'].map(lambda s : str(s).translate(translator))
  return data

In [12]:
data1 = strip_punctuations(data1)
data_top1 = data1.head()
print("Chromium strip punctuations")
data_top1

Chromium strip punctuations


,type,text
0,Bug,Define a way to enforce performance constraint...
1,Compat,Websites that dont work with Chrome Browser nC...
2,Bug,Redraw issuetruncation with confirmation item ...
3,Feature,Implement CanvasRenderingContext2D for access ...
4,Bug,ath9k panic in athtxlastbeacon 0x29a nToT 30 ...


In [13]:
data2 = strip_punctuations(data2)
data_top2 = data2.head()
print("Linux strip punctuations")
data_top2

Linux strip punctuations


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops when using idecd with 2545 and cdrecord P...
1,P2 normal,Platform Specific/Hardware,i386,NUMAQ hangs during TSC initialization on boot ...
2,P2 normal,Memory Management,Other,Enabling shared pagetables causes KDE to wierd...
3,P2 blocking,IO/Storage,Other,Dcache spirals out of control on 2543mm2 Exact...
4,P2 normal,Memory Management,Other,64GB highmem BUG Exact Kernel version 2540 Har...


**6. Remove the linux garbage in Linux Bugs data**

In [14]:
def remove_linux_garbage(data):
  '''
  Linux data contains lots of garbage, e.g. memory addresses - 0000f800
  '''
  def is_garbage(w):
    return len(w) >= 7 and sum(c.isdigit() for c in w) >= 2

  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: w if not is_garbage(w) else ' ', s.split())))
  return data
data = remove_linux_garbage(data2)
data_top2 = data2.head()
print("Remove linux garbage")
data_top2

Remove linux garbage


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops when using idecd with 2545 and cdrecord P...
1,P2 normal,Platform Specific/Hardware,i386,NUMAQ hangs during TSC initialization on boot ...
2,P2 normal,Memory Management,Other,Enabling shared pagetables causes KDE to wierd...
3,P2 blocking,IO/Storage,Other,Dcache spirals out of control on Exact Kerne...
4,P2 normal,Memory Management,Other,64GB highmem BUG Exact Kernel version 2540 Har...


**7. Naive Bayes with Chromium bugs data**

In [15]:
print('Chromium Classifying with NaiveBayes!')
def naive_bayes_classify(data):
  class_to_predict = 'type' # product importance
  all_data = [tuple(x) for x in data[['text', class_to_predict]].to_records(index=False)]

  text_counts = {}
  for item in all_data:
    for word in set(item[0].split()):
      if word in text_counts:
        text_counts[word] += 1
      else:
        text_counts[word] = 1

  for i in range(len(all_data)):
    new_text = ''
    for word in all_data[i][0].split():
      if text_counts[word] >= 5:
        new_text += ' ' + word
    all_data[i] = (new_text, all_data[i][1])

  test_corpus = all_data[3000:3600]
  print("Training Data")
  for i in test_corpus[1:6]:
    print(i)
  training_corpus = all_data[:3400]
  print("Testing Data")
  for i in training_corpus[1:6]:
    print(i)

  model = NBC(training_corpus, verbose=True)
  print('Done training!')
  print('Accuracy: ' + str(model.accuracy(test_corpus)))

  y_pred = []
  y_true = []
  for test_item in test_corpus:
    y_pred.append(model.prob_classify(test_item[0]).max())
    y_true.append(test_item[1])
  
  print('F1 score: ' + str(f1_score(y_true, y_pred, average='weighted')))

naive_bayes_classify(data1)

Chromium Classifying with NaiveBayes!
Training Data
(' Dont apply delete on shutdown rules when chrome nIf the user has selected delete on shutdown for site data appcache etc dont delete it Also dont delete the site data for origins localStorage etcrnn', 'Feature')
(' When installing extensions failure with error could not create directory for unzipping nChrome Version if applicable nOther browsers testednrnWhat steps will reproduce the problemn1When install extensions failure with error could not create directory for unzipping the singleprocess can fixed the Use the singleprocess all extensions cant use click any extensions button didnt have any', 'Bug')
(' Make an admin page for migrating or removing points with particular IDs for particular tests have been a few cases in the past when weve wanted to move all timestamp ID points to use a different version number or possible delete certain points Users of the dashboard may want to do this again in the we made an admin page that could 

**8. Naive Bayes with Linux data**

In [16]:
print('Linux Classifying with NaiveBayes!')
def naive_bayes_classify(data):
  class_to_predict = 'importance' # product importance
  all_data = [tuple(x) for x in data[['text', class_to_predict]].to_records(index=False)]

  text_counts = {}
  for item in all_data:
    for word in set(item[0].split()):
      if word in text_counts:
        text_counts[word] += 1
      else:
        text_counts[word] = 1

  for i in range(len(all_data)):
    new_text = ''
    for word in all_data[i][0].split():
      if text_counts[word] >= 5:
        new_text += ' ' + word
    all_data[i] = (new_text, all_data[i][1])

  test_corpus = all_data[3000:3600]
  print("Training Data")
  for i in test_corpus[1:6]:
    print(i)
  training_corpus = all_data[:3400]
  print("Testing Data")
  for i in training_corpus[1:6]:
    print(i)

  model = NBC(training_corpus, verbose=True)
  print('Done training!')
  print('Accuracy: ' + str(model.accuracy(test_corpus)))

  y_pred = []
  y_true = []
  for test_item in test_corpus:
    y_pred.append(model.prob_classify(test_item[0]).max())
    y_true.append(test_item[1])
  
  print('F1 score: ' + str(f1_score(y_true, y_pred, average='weighted')))
naive_bayes_classify(data2)

Linux Classifying with NaiveBayes!
Training Data
(' mouse wheel stops working after using kvm switch Distribution Gentoo Hardware Environment x86 AMD Athlon XP Logitech optical mouse Problem Description I have set up the mouse wheel in X as per usual ZAxisMapping Everything works fine until I use my switch to switch to a different PC When I switch back everythings there except the mouse wheel I loaded evdev and the input device Everytime I move the mouse a lot of garbage appears as expected for clicking However when using the wheel after switching back and forth on the KVM it remains silent for the wheel This problem does not appear under 24 in an otherwise identical environment', 'P2 low')
(' AEAMLNORETURNVALUE Asus M6N Distribution Fedora Core 2 Hardware Environment Asus M6N Software Environment Bios Problem Description ACPI complains about missing return value in the method when running cat procacpibatteryBAT0state and errors out without returning battery state information Looking i

**9. Convert string into lower case with the chromium dataset**

In [17]:
def cast_to_lowercase(data):
  data['text'] = data['text'].map(lambda s : s.lower())
  return data
data1 = cast_to_lowercase(data1)
data2 = cast_to_lowercase(data2)
data_top1 = data1.head()
print("Chromium Lowercase")
data_top1

Chromium Lowercase


,type,text
0,Bug,define a way to enforce performance constraint...
1,Compat,websites that dont work with chrome browser nc...
2,Bug,redraw issuetruncation with confirmation item ...
3,Feature,implement canvasrenderingcontext2d for access ...
4,Bug,ath9k panic in athtxlastbeacon 0x29a ntot 30 ...


**10. Convert string into lower case with the Linux dataset**

In [18]:
data_top2 = data2.head()
print("Linux Lowercase")
data_top2

Linux Lowercase


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops when using idecd with 2545 and cdrecord p...
1,P2 normal,Platform Specific/Hardware,i386,numaq hangs during tsc initialization on boot ...
2,P2 normal,Memory Management,Other,enabling shared pagetables causes kde to wierd...
3,P2 blocking,IO/Storage,Other,dcache spirals out of control on exact kerne...
4,P2 normal,Memory Management,Other,64gb highmem bug exact kernel version 2540 har...


**11. Remove the stopwords in Chromium dataset**

In [19]:
def remove_stopwords(data):
  stop_words = stopwords.words('english')
  translator = str.maketrans('', '', string.punctuation)
  stop_words = set([w.translate(translator) for w in stop_words]) # Apostrophes were removed already

  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: w if w not in stop_words else ' ', s.split())))
  return data
data1 = remove_stopwords(data1)
data2 = remove_stopwords(data2)
data_top1 = data1.head()
print("Chromium Stopword")
data_top1

Chromium Stopword


,type,text
0,Bug,define way enforce performance constraint ...
1,Compat,websites work chrome browser nchrome ver...
2,Bug,redraw issuetruncation confirmation item d...
3,Feature,implement canvasrenderingcontext2d access ...
4,Bug,ath9k panic athtxlastbeacon 0x29a ntot 30 ke...


**12. Remove the stopwords in Linux dataset**

In [20]:
data_top2 = data2.head()
print("Linux Stopword")
data_top2

Linux Stopword


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops using idecd 2545 cdrecord please en...
1,P2 normal,Platform Specific/Hardware,i386,numaq hangs tsc initialization boot exact ...
2,P2 normal,Memory Management,Other,enabling shared pagetables causes kde wierd ...
3,P2 blocking,IO/Storage,Other,dcache spirals control exact kernel vers...
4,P2 normal,Memory Management,Other,64gb highmem bug exact kernel version 2540 har...


**13. Remove the rare words in Chromium Dataset**

In [21]:
def remove_rare_words(data, min_count=3):
  wc = {} # WordCount
  def proc_word(s):
    for w in set(s.split()):
      if w in wc:
        wc[w] += 1
      else:
        wc[w] = 1

  for index, row in data.iterrows():
    proc_word(row['text'])

  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: w if wc[w] >= min_count else ' ', s.split())))
  return data
data1 = remove_rare_words(data1)
data2 = remove_rare_words(data2)
data_top1 = data1.head()
print("Chromium rare words")
data_top1

Chromium rare words


,type,text
0,Bug,define way enforce performance constraint spec...
1,Compat,websites work chrome browser nchrome version ...
2,Bug,redraw confirmation item download bar nchrom...
3,Feature,implement canvasrenderingcontext2d access work...
4,Bug,ath9k panic ntot 30 kernel resume associ...


**13. Remove the rare words in Linux Dataset**

In [22]:
data_top2 = data2.head()
print("Linux rare words")
data_top2

Linux rare words


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops using idecd 2545 cdrecord please enter ex...
1,P2 normal,Platform Specific/Hardware,i386,numaq hangs tsc initialization boot exact kern...
2,P2 normal,Memory Management,Other,enabling shared pagetables causes kde wierd ex...
3,P2 blocking,IO/Storage,Other,dcache control exact kernel version distribu...
4,P2 normal,Memory Management,Other,64gb highmem bug exact kernel version 2540 har...


**14. TF-IDF WITH SVM Chromium bugs data**

In [23]:
print("Chromium SVM Classification")
def tfidf_classify(data, model_type='SVM', extra_params={'min_df': 0.001}):
  '''
  data is a pandas dataframe
  '''
  class_to_predict = 'type' # product importance
  data = shuffle(data, random_state=77)

  num_records = len(data)
  data_train = data[:int(0.85 * num_records)]
  data_test = data[int(0.85 * num_records):]

  train_data = [x[0] for x in data_train[['text']].to_records(index=False)]
  train_labels = [x[0] for x in data_train[[class_to_predict]].to_records(index=False)]

  test_data = [x[0] for x in data_test[['text']].to_records(index=False)]
  test_labels = [x[0] for x in data_test[[class_to_predict]].to_records(index=False)]

  # Create feature vectors 
  vectorizer = TfidfVectorizer(**extra_params)
  print("Vectors")
  for i in train_data[1:6]:
      print(i)
  print("Labels")
  for i in train_labels[1:6]:
      print(i)
  # Train the feature vectors
  train_vectors = vectorizer.fit_transform(train_data)
  test_vectors = vectorizer.transform(test_data)

  # Perform classification with SVM, kernel=linear 
  if model_type == 'SVM':
    model = svm.SVC(kernel='linear')
  elif model_type == 'NN':
    model = MLPClassifier(hidden_layer_sizes=(200, 200), max_iter=4000)
  print('Training the model!')
  model.fit(train_vectors, train_labels) 
  train_prediction = model.predict(train_vectors)
  test_prediction = model.predict(test_vectors)

  train_accuracy = np.sum((np.array(train_labels) == np.array(train_prediction))) * 1.0 / len(train_labels)
  print('Training accuracy: ' + str(train_accuracy))

  test_accuracy = np.sum((np.array(test_labels) == np.array(test_prediction))) * 1.0 / len(test_labels)
  print('Test accuracy: ' + str(test_accuracy)) 

  return print('F1 score: ' + str(f1_score(test_labels, test_prediction, average='weighted')))

tfidf_classify(data1, model_type='SVM')

Chromium SVM Classification
Vectors
legacy webaudio   remove nthis tracking bug issues legacy webaudio methods attributes need removed support prefixed audiocontext removedn
gn need mechanism share parts gn config nv8 rolls srcbuild dependency gn   time eg somebody adds something build requires   whitelist v8s gn run breaks whitelisting added chromiums gn v8s instead sharing would even better whitelist files build thirdparty deps v8 pov wildcard files       change chromium side whitelisting gn existed already  
  crashes google chrome 705362 dev nchrome version 705362 devrnurls applicable   browsers testednadd ok fail browsers tested issuensafari 4n firefox 3x okrnie 7nie 8nrnwhat steps reproduce problemn1 go   expected resultn browser crashesrnrnwhat happens insteadnrnrnplease provide additional information attach screenshot ifnpossiblenn
  value gobi nogobi images nthe   x86alex builds contain hardware board value   value prevent us differentiating images via   monitoring update   n


**15. TF-IDF WITH SVM Linux bugs data**

In [24]:
print("Linux SVM Classification")
def tfidf_classify(data, model_type='SVM', extra_params={'min_df': 0.001}):
  '''
  data is a pandas dataframe
  '''
  class_to_predict = 'importance' # product importance
  data = shuffle(data, random_state=77)

  num_records = len(data)
  data_train = data[:int(0.85 * num_records)]
  data_test = data[int(0.85 * num_records):]

  train_data = [x[0] for x in data_train[['text']].to_records(index=False)]
  train_labels = [x[0] for x in data_train[[class_to_predict]].to_records(index=False)]

  test_data = [x[0] for x in data_test[['text']].to_records(index=False)]
  test_labels = [x[0] for x in data_test[[class_to_predict]].to_records(index=False)]

  print("Vectors")
  for i in train_data[1:6]:
      print(i)
  print("Labels")
  for i in train_labels[1:6]:
      print(i)
  # Create feature vectors 
  vectorizer = TfidfVectorizer(**extra_params)
  # Train the feature vectors
  train_vectors = vectorizer.fit_transform(train_data)
  test_vectors = vectorizer.transform(test_data)

  # Perform classification with SVM, kernel=linear 
  if model_type == 'SVM':
    model = svm.SVC(kernel='linear')
  elif model_type == 'NN':
    model = MLPClassifier(hidden_layer_sizes=(200, 200), max_iter=4000)
  print('Training the model!')
  model.fit(train_vectors, train_labels) 
  train_prediction = model.predict(train_vectors)
  test_prediction = model.predict(test_vectors)

  train_accuracy = np.sum((np.array(train_labels) == np.array(train_prediction))) * 1.0 / len(train_labels)
  print('Training accuracy: ' + str(train_accuracy))

  test_accuracy = np.sum((np.array(test_labels) == np.array(test_prediction))) * 1.0 / len(test_labels)
  print('Test accuracy: ' + str(test_accuracy)) 

  return print('F1 score: ' + str(f1_score(test_labels, test_prediction, average='weighted')))

tfidf_classify(data2, model_type='SVM')

Linux SVM Classification
Vectors
merlin   pcmcia card recognized serial port recent kernel bug occur 24x distribution debian hardware environment ibm laptop t42 software environment kernelorg kernel problem description insert pcmcia card log detects serial port noticed steps reproduce insert card merlin   pcmcia novatel vodafone fcc id syslog feb 7   localhost kernel pccard pcmcia card inserted slot 0 write see need install pcmciautils pcmcia utilities linux 26 reading   mentions module pcmcia 16 bit cards loaded automaticaly modprobe everything starts working please close call hope helps someone else feb 8   localhost kernel pcmcia registering new device feb 8   localhost kernel pcmcia registering new device feb 8   localhost kernel 00 ttys1 io 0x4100 irq 3 16550a
kernel bug tun device closed oops attached distribution heavily modified redhat 73 hardware environment msi motherboard athlon thunderbird 1ghz cpu 2 disks etc software environment problem description   works fine normally c

**17. Chromium Data with Non Linear-SVM Classification** 

In [25]:
print("Chromium Non Linear-SVM Classification")
extra_params={'min_df': 0.001}
data=data1;
class_to_predict = 'type' # product importance
data = shuffle(data, random_state=77)
num_records = len(data)
data_train = data[:int(0.85 * num_records)]
data_test = data[int(0.85 * num_records):]
train_data = [x[0] for x in data_train[['text']].to_records(index=False)]
train_labels = [x[0] for x in data_train[[class_to_predict]].to_records(index=False)]
test_data = [x[0] for x in data_test[['text']].to_records(index=False)]
test_labels = [x[0] for x in data_test[[class_to_predict]].to_records(index=False)]
# Create feature vectors 
vectorizer = TfidfVectorizer(**extra_params)
print("Vectors")
for i in train_data[1:6]:
  print(i)
print("Labels")
for i in train_labels[1:6]:
  print(i)
# Train the feature vectors
train_vectors = vectorizer.fit_transform(train_data)
test_vectors = vectorizer.transform(test_data)
model = svm.SVC(kernel='rbf')
print('Training the model!')
model.fit(train_vectors, train_labels) 
train_prediction = model.predict(train_vectors)
test_prediction = model.predict(test_vectors)
train_accuracy = np.sum((np.array(train_labels) == np.array(train_prediction))) * 1.0 / len(train_labels)
print('Training accuracy: ' + str(train_accuracy))
test_accuracy = np.sum((np.array(test_labels) == np.array(test_prediction))) * 1.0 / len(test_labels)
print('Test accuracy: ' + str(test_accuracy)) 
print('F1 score: ' + str(f1_score(test_labels, test_prediction, average='weighted')))

Chromium Non Linear-SVM Classification
Vectors
legacy webaudio   remove nthis tracking bug issues legacy webaudio methods attributes need removed support prefixed audiocontext removedn
gn need mechanism share parts gn config nv8 rolls srcbuild dependency gn   time eg somebody adds something build requires   whitelist v8s gn run breaks whitelisting added chromiums gn v8s instead sharing would even better whitelist files build thirdparty deps v8 pov wildcard files       change chromium side whitelisting gn existed already  
  crashes google chrome 705362 dev nchrome version 705362 devrnurls applicable   browsers testednadd ok fail browsers tested issuensafari 4n firefox 3x okrnie 7nie 8nrnwhat steps reproduce problemn1 go   expected resultn browser crashesrnrnwhat happens insteadnrnrnplease provide additional information attach screenshot ifnpossiblenn
  value gobi nogobi images nthe   x86alex builds contain hardware board value   value prevent us differentiating images via   monitoring 

**18.Linux data with Non Linear-SVM Classification**

In [26]:
print("Linux Non Linear-SVM Classification")
extra_params={'min_df': 0.001}
data=data2;
class_to_predict = 'importance' # product importance
data = shuffle(data, random_state=77)
num_records = len(data)
data_train = data[:int(0.85 * num_records)]
data_test = data[int(0.85 * num_records):]
train_data = [x[0] for x in data_train[['text']].to_records(index=False)]
train_labels = [x[0] for x in data_train[[class_to_predict]].to_records(index=False)]
test_data = [x[0] for x in data_test[['text']].to_records(index=False)]
test_labels = [x[0] for x in data_test[[class_to_predict]].to_records(index=False)]
# Create feature vectors 
vectorizer = TfidfVectorizer(**extra_params)
print("Vectors")
for i in train_data[1:6]:
  print(i)
print("Labels")
for i in train_labels[1:6]:
  print(i)
# Train the feature vectors
train_vectors = vectorizer.fit_transform(train_data)
test_vectors = vectorizer.transform(test_data)
model = svm.SVC(kernel='rbf')
print('Training the model!')
model.fit(train_vectors, train_labels) 
train_prediction = model.predict(train_vectors)
test_prediction = model.predict(test_vectors)
train_accuracy = np.sum((np.array(train_labels) == np.array(train_prediction))) * 1.0 / len(train_labels)
print('Training accuracy: ' + str(train_accuracy))
test_accuracy = np.sum((np.array(test_labels) == np.array(test_prediction))) * 1.0 / len(test_labels)
print('Test accuracy: ' + str(test_accuracy)) 
print('F1 score: ' + str(f1_score(test_labels, test_prediction, average='weighted')))

Linux Non Linear-SVM Classification
Vectors
merlin   pcmcia card recognized serial port recent kernel bug occur 24x distribution debian hardware environment ibm laptop t42 software environment kernelorg kernel problem description insert pcmcia card log detects serial port noticed steps reproduce insert card merlin   pcmcia novatel vodafone fcc id syslog feb 7   localhost kernel pccard pcmcia card inserted slot 0 write see need install pcmciautils pcmcia utilities linux 26 reading   mentions module pcmcia 16 bit cards loaded automaticaly modprobe everything starts working please close call hope helps someone else feb 8   localhost kernel pcmcia registering new device feb 8   localhost kernel pcmcia registering new device feb 8   localhost kernel 00 ttys1 io 0x4100 irq 3 16550a
kernel bug tun device closed oops attached distribution heavily modified redhat 73 hardware environment msi motherboard athlon thunderbird 1ghz cpu 2 disks etc software environment problem description   works fine

**19.Chromium data with SVM Poly Classification**

In [27]:
print("Chromium SVM Poly Classification")
def tfidf_classify(data, model_type='SVM', extra_params={'min_df': 0.001}):
  '''
  data is a pandas dataframe
  '''
  class_to_predict = 'type' # product importance
  data = shuffle(data, random_state=77)

  num_records = len(data)
  data_train = data[:int(0.85 * num_records)]
  data_test = data[int(0.85 * num_records):]

  train_data = [x[0] for x in data_train[['text']].to_records(index=False)]
  train_labels = [x[0] for x in data_train[[class_to_predict]].to_records(index=False)]

  test_data = [x[0] for x in data_test[['text']].to_records(index=False)]
  test_labels = [x[0] for x in data_test[[class_to_predict]].to_records(index=False)]

  # Create feature vectors 
  vectorizer = TfidfVectorizer(**extra_params)
  print("Vectors")
  for i in train_data[1:6]:
      print(i)
  print("Labels")
  for i in train_labels[1:6]:
      print(i)
  # Train the feature vectors
  train_vectors = vectorizer.fit_transform(train_data)
  test_vectors = vectorizer.transform(test_data)

  # Perform classification with SVM, kernel=polynomial
  if model_type == 'SVM':
    model = svm.SVC(kernel='poly')
  print('Training the model!')
  model.fit(train_vectors, train_labels) 
  train_prediction = model.predict(train_vectors)
  test_prediction = model.predict(test_vectors)

  train_accuracy = np.sum((np.array(train_labels) == np.array(train_prediction))) * 1.0 / len(train_labels)
  print('Training accuracy: ' + str(train_accuracy))

  test_accuracy = np.sum((np.array(test_labels) == np.array(test_prediction))) * 1.0 / len(test_labels)
  print('Test accuracy: ' + str(test_accuracy)) 

  return print('F1 score: ' + str(f1_score(test_labels, test_prediction, average='weighted')))

tfidf_classify(data1, model_type='SVM')

Chromium SVM Poly Classification
Vectors
legacy webaudio   remove nthis tracking bug issues legacy webaudio methods attributes need removed support prefixed audiocontext removedn
gn need mechanism share parts gn config nv8 rolls srcbuild dependency gn   time eg somebody adds something build requires   whitelist v8s gn run breaks whitelisting added chromiums gn v8s instead sharing would even better whitelist files build thirdparty deps v8 pov wildcard files       change chromium side whitelisting gn existed already  
  crashes google chrome 705362 dev nchrome version 705362 devrnurls applicable   browsers testednadd ok fail browsers tested issuensafari 4n firefox 3x okrnie 7nie 8nrnwhat steps reproduce problemn1 go   expected resultn browser crashesrnrnwhat happens insteadnrnrnplease provide additional information attach screenshot ifnpossiblenn
  value gobi nogobi images nthe   x86alex builds contain hardware board value   value prevent us differentiating images via   monitoring update

**20.Linux SVM Poly Classification**

In [28]:
print("Linux SVM Poly Classification")
def tfidf_classify(data, model_type='SVM', extra_params={'min_df': 0.001}):
  '''
  data is a pandas dataframe
  '''
  class_to_predict = 'importance' # product importance
  data = shuffle(data, random_state=77)

  num_records = len(data)
  data_train = data[:int(0.85 * num_records)]
  data_test = data[int(0.85 * num_records):]

  train_data = [x[0] for x in data_train[['text']].to_records(index=False)]
  train_labels = [x[0] for x in data_train[[class_to_predict]].to_records(index=False)]

  test_data = [x[0] for x in data_test[['text']].to_records(index=False)]
  test_labels = [x[0] for x in data_test[[class_to_predict]].to_records(index=False)]

  # Create feature vectors 
  vectorizer = TfidfVectorizer(**extra_params)
  print("Vectors")
  for i in train_data[1:6]:
      print(i)
  print("Labels")
  for i in train_labels[1:6]:
      print(i)
  # Train the feature vectors
  train_vectors = vectorizer.fit_transform(train_data)
  test_vectors = vectorizer.transform(test_data)

  # Perform classification with SVM, kernel=polynomial
  if model_type == 'SVM':
    model = svm.SVC(kernel='poly')
  print('Training the model!')
  model.fit(train_vectors, train_labels) 
  train_prediction = model.predict(train_vectors)
  test_prediction = model.predict(test_vectors)

  train_accuracy = np.sum((np.array(train_labels) == np.array(train_prediction))) * 1.0 / len(train_labels)
  print('Training accuracy: ' + str(train_accuracy))

  test_accuracy = np.sum((np.array(test_labels) == np.array(test_prediction))) * 1.0 / len(test_labels)
  print('Test accuracy: ' + str(test_accuracy)) 

  return print('F1 score: ' + str(f1_score(test_labels, test_prediction, average='weighted')))

tfidf_classify(data2, model_type='SVM')

Linux SVM Poly Classification
Vectors
merlin   pcmcia card recognized serial port recent kernel bug occur 24x distribution debian hardware environment ibm laptop t42 software environment kernelorg kernel problem description insert pcmcia card log detects serial port noticed steps reproduce insert card merlin   pcmcia novatel vodafone fcc id syslog feb 7   localhost kernel pccard pcmcia card inserted slot 0 write see need install pcmciautils pcmcia utilities linux 26 reading   mentions module pcmcia 16 bit cards loaded automaticaly modprobe everything starts working please close call hope helps someone else feb 8   localhost kernel pcmcia registering new device feb 8   localhost kernel pcmcia registering new device feb 8   localhost kernel 00 ttys1 io 0x4100 irq 3 16550a
kernel bug tun device closed oops attached distribution heavily modified redhat 73 hardware environment msi motherboard athlon thunderbird 1ghz cpu 2 disks etc software environment problem description   works fine norma

**21. Fast text with Chromium bugs data**

In [29]:
# These are the optimal parameters for the 'importance' prediction
# For other columns they are different.
print("Chromium FastText Classification")
TRAIN_PATH = './fasttext_train.txt'
TEST_PATH = './fasttext_test.txt'
MODEL_PATH = './ft.model'

def fasttext_classify(data, extra_params={}):
  class_to_predict = 'type' # product importance
  data[class_to_predict] = data[class_to_predict].map(lambda s : s.replace(" ", ""))
  data_for_fasttext = data['text'] + ' __label__' + data[class_to_predict]
  data_for_fasttext = shuffle(data_for_fasttext, random_state=77)

  num_records = len(data_for_fasttext)
  data_train = data_for_fasttext[:int(0.85 * num_records)]
  data_test = data_for_fasttext[int(0.85 * num_records):]
  print("Train data")
  data_top2 = data_for_fasttext.head()
  data_top2
  data_train.to_csv(TRAIN_PATH, sep='\t', header=0, index=False)
  data_test.to_csv(TEST_PATH, sep='\t', header=0, index=False)

  model = fasttext.train_supervised(TRAIN_PATH, **extra_params)
  #model.save_model(MODEL_PATH)
  print('Training accuracy:')
  train_accuracy = model.test(TRAIN_PATH)
  print(train_accuracy[-1])

  print('Test accuracy:')
  test_accuracy = model.test(TEST_PATH)
  print(test_accuracy[-1])

  y_pred = []
  y_true = []
  for test_item in data_test:
    test_text, test_label = test_item.split('__label__')
    y_pred.append(model.predict(test_text)[0])
    y_true.append('__label__' + test_label)

  return print('F1 score: ' + str(f1_score(y_true, y_pred, average='weighted'))) # accuracy is a tuple

fasttext_classify(data1, extra_params={
  'epoch': 15,
  'minCount': 1,
  'dim': 150,
  'ws': 5,
  'neg': 5,
  'wordNgrams': 2,
  'verbose': 2
})

Chromium FastText Classification
Train data
Training accuracy:
0.998701039168665
Test accuracy:
0.8201789151851433
F1 score: 0.8200254198017213


**22. Fast text with Linux data**

In [30]:
print("Linux FastText Classification")
TRAIN_PATH = './fasttext_train.txt'
TEST_PATH = './fasttext_test.txt'
MODEL_PATH = './ft.model'

def fasttext_classify(data, extra_params={}):
  class_to_predict = 'importance' # product importance
  data[class_to_predict] = data[class_to_predict].map(lambda s : s.replace(" ", ""))
  data_for_fasttext = data['text'] + ' __label__' + data[class_to_predict]
  data_for_fasttext = shuffle(data_for_fasttext, random_state=77)

  num_records = len(data_for_fasttext)
  data_train = data_for_fasttext[:int(0.85 * num_records)]
  data_test = data_for_fasttext[int(0.85 * num_records):]
  print("Train data")
  data_top2 = data_for_fasttext.head()
  data_top2
  data_train.to_csv(TRAIN_PATH, sep='\t', header=0, index=False)
  data_test.to_csv(TEST_PATH, sep='\t', header=0, index=False)

  model = fasttext.train_supervised(TRAIN_PATH, **extra_params)
  #model.save_model(MODEL_PATH)
  print('Training accuracy:')
  train_accuracy = model.test(TRAIN_PATH)
  print(train_accuracy[-1])

  print('Test accuracy:')
  test_accuracy = model.test(TEST_PATH)
  print(test_accuracy[-1])

  y_pred = []
  y_true = []
  for test_item in data_test:
    test_text, test_label = test_item.split('__label__')
    y_pred.append(model.predict(test_text)[0])
    y_true.append('__label__' + test_label)

  return print('F1 score: ' + str(f1_score(y_true, y_pred, average='weighted')))# accuracy is a tuple
fasttext_classify(data2, extra_params={
  'epoch': 15,
  'minCount': 1,
  'dim': 150,
  'ws': 5,
  'neg': 5,
  'wordNgrams': 2,
  'verbose': 2
})

Linux FastText Classification
Train data
Training accuracy:
0.7267462643883607
Test accuracy:
0.6423653300931551
F1 score: 0.5583709963707842


**23. Decision Tree with the chromium Dataset**




In [31]:
print("Chromium DT Classification")
from sklearn.tree import DecisionTreeClassifier
def dt_classify(data,extra_params={'min_df': 0.001}):
  '''
  data is a pandas dataframe
  '''
  class_to_predict = 'type' # product importance
  data = shuffle(data, random_state=77)

  num_records = len(data)
  data_train = data[:int(0.85 * num_records)]
  data_test = data[int(0.85 * num_records):]

  train_data = [x[0] for x in data_train[['text']].to_records(index=False)]
  train_labels = [x[0] for x in data_train[[class_to_predict]].to_records(index=False)]

  test_data = [x[0] for x in data_test[['text']].to_records(index=False)]
  test_labels = [x[0] for x in data_test[[class_to_predict]].to_records(index=False)]

  print("Vectors")
  for i in train_data[1:6]:
      print(i)
  print("Labels")
  for i in train_labels[1:6]:
      print(i)
  # Create feature vectors 
  vectorizer = TfidfVectorizer(**extra_params)
  # Train the feature vectors
  train_vectors = vectorizer.fit_transform(train_data)
  test_vectors = vectorizer.transform(test_data)

  # Perform classification with SVM, kernel=linear 
  model = DecisionTreeClassifier(random_state=0)
  print('Training the model!')
  model.fit(train_vectors, train_labels) 
  train_prediction = model.predict(train_vectors)
  test_prediction = model.predict(test_vectors)

  train_accuracy = np.sum((np.array(train_labels) == np.array(train_prediction))) * 1.0 / len(train_labels)
  print('Training accuracy: ' + str(train_accuracy))

  test_accuracy = np.sum((np.array(test_labels) == np.array(test_prediction))) * 1.0 / len(test_labels)
  print('Test accuracy: ' + str(test_accuracy)) 

  return print('F1 score: ' + str(f1_score(test_labels, test_prediction, average='weighted')))

dt_classify(data1)

Chromium DT Classification
Vectors
legacy webaudio   remove nthis tracking bug issues legacy webaudio methods attributes need removed support prefixed audiocontext removedn
gn need mechanism share parts gn config nv8 rolls srcbuild dependency gn   time eg somebody adds something build requires   whitelist v8s gn run breaks whitelisting added chromiums gn v8s instead sharing would even better whitelist files build thirdparty deps v8 pov wildcard files       change chromium side whitelisting gn existed already  
  crashes google chrome 705362 dev nchrome version 705362 devrnurls applicable   browsers testednadd ok fail browsers tested issuensafari 4n firefox 3x okrnie 7nie 8nrnwhat steps reproduce problemn1 go   expected resultn browser crashesrnrnwhat happens insteadnrnrnplease provide additional information attach screenshot ifnpossiblenn
  value gobi nogobi images nthe   x86alex builds contain hardware board value   value prevent us differentiating images via   monitoring update   n
a

**24. Decision Tree with the Linux Dataset**

In [32]:
print("Linux DT Classification")
from sklearn.tree import DecisionTreeClassifier
def dt_classify(data,extra_params={'min_df': 0.001}):
  '''
  data is a pandas dataframe
  '''
  class_to_predict = 'importance' # product importance
  data = shuffle(data, random_state=77)

  num_records = len(data)
  data_train = data[:int(0.85 * num_records)]
  data_test = data[int(0.85 * num_records):]

  train_data = [x[0] for x in data_train[['text']].to_records(index=False)]
  train_labels = [x[0] for x in data_train[[class_to_predict]].to_records(index=False)]

  test_data = [x[0] for x in data_test[['text']].to_records(index=False)]
  test_labels = [x[0] for x in data_test[[class_to_predict]].to_records(index=False)]

  print("Vectors")
  for i in train_data[1:6]:
      print(i)
  print("Labels")
  for i in train_labels[1:6]:
      print(i)
  # Create feature vectors 
  vectorizer = TfidfVectorizer(**extra_params)
  # Train the feature vectors
  train_vectors = vectorizer.fit_transform(train_data)
  test_vectors = vectorizer.transform(test_data)

  # Perform classification with SVM, kernel=linear 
  model = DecisionTreeClassifier(random_state=0)
  print('Training the model!')
  model.fit(train_vectors, train_labels) 
  train_prediction = model.predict(train_vectors)
  test_prediction = model.predict(test_vectors)

  train_accuracy = np.sum((np.array(train_labels) == np.array(train_prediction))) * 1.0 / len(train_labels)
  print('Training accuracy: ' + str(train_accuracy))

  test_accuracy = np.sum((np.array(test_labels) == np.array(test_prediction))) * 1.0 / len(test_labels)
  print('Test accuracy: ' + str(test_accuracy)) 

  return print('F1 score: ' + str(f1_score(test_labels, test_prediction, average='weighted')))

dt_classify(data2)

Linux DT Classification
Vectors
merlin   pcmcia card recognized serial port recent kernel bug occur 24x distribution debian hardware environment ibm laptop t42 software environment kernelorg kernel problem description insert pcmcia card log detects serial port noticed steps reproduce insert card merlin   pcmcia novatel vodafone fcc id syslog feb 7   localhost kernel pccard pcmcia card inserted slot 0 write see need install pcmciautils pcmcia utilities linux 26 reading   mentions module pcmcia 16 bit cards loaded automaticaly modprobe everything starts working please close call hope helps someone else feb 8   localhost kernel pcmcia registering new device feb 8   localhost kernel pcmcia registering new device feb 8   localhost kernel 00 ttys1 io 0x4100 irq 3 16550a
kernel bug tun device closed oops attached distribution heavily modified redhat 73 hardware environment msi motherboard athlon thunderbird 1ghz cpu 2 disks etc software environment problem description   works fine normally cl

**25. Random forest with the Chromium Dataset**

In [33]:
print("Chromium RF Classification")
from sklearn.ensemble import RandomForestClassifier
def rf_classify(data,extra_params={'min_df': 0.001}):
  '''
  data is a pandas dataframe
  '''
  class_to_predict = 'type' # product importance
  data = shuffle(data, random_state=77)

  num_records = len(data)
  data_train = data[:int(0.85 * num_records)]
  data_test = data[int(0.85 * num_records):]

  train_data = [x[0] for x in data_train[['text']].to_records(index=False)]
  train_labels = [x[0] for x in data_train[[class_to_predict]].to_records(index=False)]

  test_data = [x[0] for x in data_test[['text']].to_records(index=False)]
  test_labels = [x[0] for x in data_test[[class_to_predict]].to_records(index=False)]

  print("Vectors")
  for i in train_data[1:6]:
      print(i)
  print("Labels")
  for i in train_labels[1:6]:
      print(i)
  # Create feature vectors 
  vectorizer = TfidfVectorizer(**extra_params)
  # Train the feature vectors
  train_vectors = vectorizer.fit_transform(train_data)
  test_vectors = vectorizer.transform(test_data)

  # Perform classification with SVM, kernel=linear 
  model = RandomForestClassifier(max_depth=2, random_state=0)
  print('Training the model!')
  model.fit(train_vectors, train_labels) 
  train_prediction = model.predict(train_vectors)
  test_prediction = model.predict(test_vectors)

  train_accuracy = np.sum((np.array(train_labels) == np.array(train_prediction))) * 1.0 / len(train_labels)
  print('Training accuracy: ' + str(train_accuracy))

  test_accuracy = np.sum((np.array(test_labels) == np.array(test_prediction))) * 1.0 / len(test_labels)
  print('Test accuracy: ' + str(test_accuracy)) 

  return print('F1 score: ' + str(f1_score(test_labels, test_prediction, average='weighted')))

rf_classify(data1)

Chromium RF Classification
Vectors
legacy webaudio   remove nthis tracking bug issues legacy webaudio methods attributes need removed support prefixed audiocontext removedn
gn need mechanism share parts gn config nv8 rolls srcbuild dependency gn   time eg somebody adds something build requires   whitelist v8s gn run breaks whitelisting added chromiums gn v8s instead sharing would even better whitelist files build thirdparty deps v8 pov wildcard files       change chromium side whitelisting gn existed already  
  crashes google chrome 705362 dev nchrome version 705362 devrnurls applicable   browsers testednadd ok fail browsers tested issuensafari 4n firefox 3x okrnie 7nie 8nrnwhat steps reproduce problemn1 go   expected resultn browser crashesrnrnwhat happens insteadnrnrnplease provide additional information attach screenshot ifnpossiblenn
  value gobi nogobi images nthe   x86alex builds contain hardware board value   value prevent us differentiating images via   monitoring update   n
a

**26. Random forest with the Linux Dataset**

In [34]:
print("Linux RF Classification")
from sklearn.ensemble import RandomForestClassifier
def rf_classify(data,extra_params={'min_df': 0.001}):
  '''
  data is a pandas dataframe
  '''
  class_to_predict = 'importance' # product importance
  data = shuffle(data, random_state=77)

  num_records = len(data)
  data_train = data[:int(0.85 * num_records)]
  data_test = data[int(0.85 * num_records):]

  train_data = [x[0] for x in data_train[['text']].to_records(index=False)]
  train_labels = [x[0] for x in data_train[[class_to_predict]].to_records(index=False)]

  test_data = [x[0] for x in data_test[['text']].to_records(index=False)]
  test_labels = [x[0] for x in data_test[[class_to_predict]].to_records(index=False)]

  print("Vectors")
  for i in train_data[1:6]:
      print(i)
  print("Labels")
  for i in train_labels[1:6]:
      print(i)
  # Create feature vectors 
  vectorizer = TfidfVectorizer(**extra_params)
  # Train the feature vectors
  train_vectors = vectorizer.fit_transform(train_data)
  test_vectors = vectorizer.transform(test_data)

  # Perform classification with SVM, kernel=linear 
  model = RandomForestClassifier(max_depth=2, random_state=0)
  print('Training the model!')
  model.fit(train_vectors, train_labels) 
  train_prediction = model.predict(train_vectors)
  test_prediction = model.predict(test_vectors)

  train_accuracy = np.sum((np.array(train_labels) == np.array(train_prediction))) * 1.0 / len(train_labels)
  print('Training accuracy: ' + str(train_accuracy))

  test_accuracy = np.sum((np.array(test_labels) == np.array(test_prediction))) * 1.0 / len(test_labels)
  print('Test accuracy: ' + str(test_accuracy)) 

  return print('F1 score: ' + str(f1_score(test_labels, test_prediction, average='weighted')))

rf_classify(data2)

Linux RF Classification
Vectors
merlin   pcmcia card recognized serial port recent kernel bug occur 24x distribution debian hardware environment ibm laptop t42 software environment kernelorg kernel problem description insert pcmcia card log detects serial port noticed steps reproduce insert card merlin   pcmcia novatel vodafone fcc id syslog feb 7   localhost kernel pccard pcmcia card inserted slot 0 write see need install pcmciautils pcmcia utilities linux 26 reading   mentions module pcmcia 16 bit cards loaded automaticaly modprobe everything starts working please close call hope helps someone else feb 8   localhost kernel pcmcia registering new device feb 8   localhost kernel pcmcia registering new device feb 8   localhost kernel 00 ttys1 io 0x4100 irq 3 16550a
kernel bug tun device closed oops attached distribution heavily modified redhat 73 hardware environment msi motherboard athlon thunderbird 1ghz cpu 2 disks etc software environment problem description   works fine normally cl